<a href="https://colab.research.google.com/github/rosacarla/Praticas-em-PLN/blob/main/Question_answering_com_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Question-answering com o BERT
## Tecnólogo em Inteligência Artificial Aplicada - Agentes Conversacionais
Neste notebook iremos construir um agente de diálogo apoiado por modelos de linguagem avançados, baseados na arquitetura Transformers.

### Qual o contexto de nosso agente?

Iremos desenvolver um agente de diálogo que responda perguntas do usuário sobre determinado tema, utilizando-se de uma API de acesso aos dados da Wikipedia e o um modelo de linguagem baseado em BERT.

### Construindo o agente de diálogo
Nosso agente vai operar da seguinte maneira:

1.   Recebe entrada do usuário
2.   Busca temas associados à paginas na Wikipedia
3.   Obtém conteúdo associado ao tema
4.   Recebe pergunta sobre o tema
5.   Utiliza modelo BERT para procurar por resposta no texto da Wikipedia

#### Instalando dependências

In [ ]:
# É preciso instalar a biblioteca da arquitetura Transformers no Google Colab - e em seu ambiente também caso você execute localmente
!pip install transformers

In [2]:
# É preciso instalar a API de acesso à Wikipedia para facilitar nossas buscas
!pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=5f9a151df6459d27859b16c04542c03145d73a90c9391da403e451b6de6e6f3d
  Stored in directory: /root/.cache/pip/wheels/8f/ab/cb/45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built wikipedia


#### Importando as bibliotecas

In [3]:
import transformers
from transformers import pipeline
import wikipedia
# Define idioma padrão
wikipedia.set_lang("pt")

#### Carrega modelo de question-answering do BERT para Português
O modelo está disponível no [site da HuggingFace](https://huggingface.co/pierreguillou/bert-base-cased-squad-v1.1-portuguese), empresa mantenedora da biblioteca Transformers.

Informações sobre como este modelo foi desenvolvido estão disponíveis [neste artigo](https://medium.com/@pierre_guillou/nlp-modelo-de-question-answering-em-qualquer-idioma-baseado-no-bert-base-estudo-de-caso-em-12093d385e78).

In [4]:
# Define o TIPO e NOME do modelo que deseja-se carregar - o nome é definido no próprio site da Hugging Face
nlp = pipeline("question-answering", model='pierreguillou/bert-base-cased-squad-v1.1-portuguese')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/862 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/433M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/494 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/210k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu


#### Função que obtém dados sobre o tema e responde às perguntas

In [5]:
def getResposta(entrada):

  # Busca página relacionada ao tema informado pelo usuário - obtém apenas a primeira ocorrência
  tema = wikipedia.search(entrada)[0]

  # Obtém todo conteúdo da página
  contexto = wikipedia.page(tema).content

  while True:

    # Pede que usuário faça pergunta sobre o tema
    print("Agente Transformers: Faça-me uma pergunta sobre o tema {} ou digite 'voltar' para alterar o tema.".format(tema))
    pergunta = input().lower()

    # Se usuário pedir para sair
    if pergunta == 'voltar':
      print("Agente Transformers: Sobre qual assunto você deseja falar?")
      return None

    # Chama o pipeline dos Transformers - que já está com o modelo de question-answering carregado
    resultado = nlp(question=pergunta, context=contexto)

    print(f"Agente Transformers: '{resultado['answer']}'")
    # Imprime score e dados adicionais
    #print(f"Score: {round(resultado['score'], 4)}, start: {resultado['start']}, end: {resultado['end']}")



#### Interagindo com o agente de diálogo
O modelo pode demorar para responder, de acordo com o tamanho do contexto (i.e., página da Wikipedia).

Exemplos de perguntas sobre "**Pandemia de COVID-19**":
* O que é a pandemia de Covid-19?
* A Covid-19 tem algo a ver com animais?
* Onde foi descoberta a Covid-19?
* Quantas mortes?

Exemplos de perguntas sobre "**Pelé**":
* Quantos gols Pelé fez?
* Onde ele nasceu?
* Por quais times ele jogou?

Exemplos de perguntas sobre "**Python**":
* Quem foi o criador do Python?
* Qual a filosofia da linguagem
* Qual a origem do nome?


In [6]:
continue_dialogue = True
saudacao = "Olá, eu sou o Agente Transformers. Sobre qual assunto você deseja falar?"
print(saudacao)
while (continue_dialogue == True):
  # Obtém entrada do usuário
  entrada = input().lower()

  if entrada not in ['tchau','adeus','até mais', 'até a próxima']:
    if entrada in ['obrigado','muito obrigado','agradecido']:
      continue_dialogue = False
      print("Agente Transformers: Disponha")
    else:
      if entrada in ['oi','oie','olá','e aí','bom dia','boa tarde', 'boa noite']:
        print("Agente Transformers: " + saudacao)
      else:
        # Caso não acione intenções de SAUDAÇÃO, AGRADECIMENTO ou DESPEDIDA
        # Entra em intenção principal - Tema e respostas
        getResposta(entrada)
  else:
    continue_dialogue = False
    print("Agente Transformers: Até a próxima.")

Olá, eu sou o Agente Transformers. Sobre qual assunto você deseja falar?
Python
Agente Transformers: Faça-me uma pergunta sobre o tema Python ou digite 'voltar' para alterar o tema.
como criar uma interface de sistem com Python?
Agente Transformers: 'substituindo blocos if-else'
Agente Transformers: Faça-me uma pergunta sobre o tema Python ou digite 'voltar' para alterar o tema.
Como usa o comando sum? 
Agente Transformers: 'Interface de linha de comandos'
Agente Transformers: Faça-me uma pergunta sobre o tema Python ou digite 'voltar' para alterar o tema.
voce demora para responder, vou voltar
Agente Transformers: 'Programa Olá Mundo'
Agente Transformers: Faça-me uma pergunta sobre o tema Python ou digite 'voltar' para alterar o tema.
voltar
Agente Transformers: Sobre qual assunto você deseja falar?


KeyboardInterrupt: Interrupted by user

### O que pode ser feito agora?

* Poderíamos utilizar o RASA ou o DialogFlow para implementar de maneira mais fácil o gerenciamento de diálogo e NLU de nosso agente, e customizar o acesso aos Tranformers utilizando actions customizadas (RASA) ou fullfilments (DialogFlow).
* Fazer desambiguação de temas sugeridos pelo usuário - a função search() retorna uma lista de páginas de wikipedia possíveis
* Tentar utilizar outros modelos baseados em Transformers ou Machine Learning em geral, como [este](https://huggingface.co/mrm8488/bert-base-portuguese-cased-finetuned-squad-v1-pt).


## Referências e Material complementar

* [NLP | Modelo de Question Answering em qualquer idioma baseado no BERT base (estudo de caso em português)](https://medium.com/@pierre_guillou/nlp-modelo-de-question-answering-em-qualquer-idioma-baseado-no-bert-base-estudo-de-caso-em-12093d385e78)
* [Hugging Face - Portuguese BERT base cased QA (Question Answering), finetuned on SQUAD v1.1](https://huggingface.co/pierreguillou/bert-base-cased-squad-v1.1-portuguese)
* [The Illustrated BERT, ELMo, and co. (How NLP Cracked Transfer Learning)](http://jalammar.github.io/illustrated-bert/)

Este notebook foi produzido por Prof. [Lucas Oliveira](http://lattes.cnpq.br/3611246009892500).